In [1]:
%cd ..

/tf/notebooks


In [2]:
import json
import re

In [3]:
with open("data/recipes/amounts.json", encoding="utf-8") as f:
    amounts = json.load(f)

In [4]:
re1 = re.compile(r"^([\d½¼¾⅛]+(?: [½¼¾⅛])?(?:,\d+)?)(?: ((?:kl\. |gr\. )?\w+(?:, gestr\.|, gehäuft)?)(?:/n|/e|\(n\)|\(e\)|\.)?)?$")
re_singol = re.compile(r"^(kl. |gr. )?(Tasse|mg|ml|Tüte|Kopf|Stück|Spritzer|Teil|Becher|Blatt|Zehe|Blätter|Bund|Dose|[ET]L(,.*)?|cl|dl|Flasche|Fläschchen|Glas|Gläser|Handvoll|Körner|Liter|Msp.|Paar|Paket|Pck.|Pkt.|Port.|Prise.*|Scheibe)(/[en])?$")


In [5]:
replacings = "gestr.=gestrichen,geh.=gehäuft,m.-große=mittelgroße,m.-großer=mittelgroßer,TL=Teelöffel,Eßlöffel=Esslöffel,EL=Esslöffel,Msp.=Messerspitze,Pck.=Packung,Pkt.=Packung,Port.=Portion,kl.=kleine,gr.=große".split(",")

replacings = {ee.split("=")[0]: ee.split("=")[1] for ee in replacings}

In [6]:
parsed1 = []
for _amount in amounts:
    amount = _amount
    if amount == "":
        amount = "1"
    #amount = amount.replace("m.-große", "mittelgroße")
    if re_singol.match(amount):
        amount = "1 " + amount
    for bef, aft in replacings.items():
        amount = re.sub(r"\b" + re.escape(bef) + r"(\b|$)", aft, amount)
    m = re1.match(amount)
    if m is not None:
        parsed1.append((_amount, *m.groups()))
    else:
        print("UNMATCHED  ", _amount, amount)
        parsed1.append((_amount, None, None))

UNMATCHED   0,2 TL, gestr. 0,2 Teelöffel, gestrichen
UNMATCHED   0,3 TL, gestr. 0,3 Teelöffel, gestrichen
UNMATCHED   0,33 EL, gestr. 0,33 Esslöffel, gestrichen
UNMATCHED   0,33 TL, gestr. 0,33 Teelöffel, gestrichen
UNMATCHED   1 EL, gestr. 1 Esslöffel, gestrichen
UNMATCHED   1 TL, gestr. 1 Teelöffel, gestrichen
UNMATCHED   1 einige Stiele 1 einige Stiele
UNMATCHED   1 m.-großes 1 m.-großes
UNMATCHED   1 n. B. 1 n. B.
UNMATCHED   1 ½ EL, gestr. 1 ½ Esslöffel, gestrichen
UNMATCHED   1 ½ TL, gestr. 1 ½ Teelöffel, gestrichen
UNMATCHED   10 EL, gestr. 10 Esslöffel, gestrichen
UNMATCHED   10 TL, gestr. 10 Teelöffel, gestrichen
UNMATCHED   10 einige Stiele 10 einige Stiele
UNMATCHED   10 n. B. 10 n. B.
UNMATCHED   12 TL, gestr. 12 Teelöffel, gestrichen
UNMATCHED   125 n. B. 125 n. B.
UNMATCHED   14 TL, gestr. 14 Teelöffel, gestrichen
UNMATCHED   15 EL, gestr. 15 Esslöffel, gestrichen
UNMATCHED   18 EL, gestr. 18 Esslöffel, gestrichen
UNMATCHED   2 EL, gestr. 2 Esslöffel, gestrichen
UNMATCHED

In [7]:
len(parsed1), len(amounts)

(2782, 2782)

In [8]:
parsed1[100]

('0,33 Tasse', '0,33', 'Tasse')

In [9]:
re_num = re.compile(r"([\d½¼¾⅛]+)(?: ([½¼¾⅛]))?(?:,(\d+))?")

In [10]:
def parse_num(amount, unit):
    if amount is None:
        return (None, None)
    fs = {"½": 1/2, "¼": 1/4, "¾": 3/4, "⅛": 1/8}
    whole, addition, fraction = re_num.match(amount).groups()
    whole = fs[whole] if whole in fs else int(whole)
    addition = fs[addition] if addition is not None else 0
    fraction = float("0." + fraction) if fraction is not None else 0
    total_amount = whole + addition + fraction
    if unit == "kg":
        unit = "g"
        total_amount *= 1000
    if unit == "mg":
        unit = "g"
        total_amount /= 1000
    if unit and (unit.lower() == "l" or unit.lower() == "liter"):
        unit = "ml"
        total_amount *= 1000
    if unit == "cl":
        unit = "ml"
        total_amount *= 10
    if unit == "dl":
        unit = "ml"
        total_amount *= 100
    return total_amount, unit

parsed_amounts = {orig: parse_num(n, e) for orig, n, e in parsed1}

In [11]:
parsed_amounts

{'': (1, None),
 '760 ml': (760, 'ml'),
 '8 Gläser': (8, 'Gläser'),
 '½ Wurzel': (0.5, 'Wurzel'),
 '4500 g': (4500, 'g'),
 '105 ml': (105, 'ml'),
 '55 ml': (55, 'ml'),
 '10 Scheibe/n': (10, 'Scheibe'),
 '365 g': (365, 'g'),
 '7 Wurzel/n': (7, 'Wurzel'),
 '1 evtl.': (1, 'evtl'),
 '8 Pkt.': (8, 'Packung'),
 '15 Körner': (15, 'Körner'),
 '1 cl': (10, 'ml'),
 '8 cl': (80, 'ml'),
 '2 cl': (20, 'ml'),
 '1 Zweig/e': (1, 'Zweig'),
 '6 kleine': (6, 'kleine'),
 '310 ml': (310, 'ml'),
 '24 m.-große': (24, 'mittelgroße'),
 '0,3': (0.3, None),
 '4 Blatt': (4, 'Blatt'),
 '585 g': (585, 'g'),
 '4 dl': (400, 'ml'),
 '710 g': (710, 'g'),
 '½ Bund': (0.5, 'Bund'),
 '0,33 kl. Glas': (0.33, 'kl. Glas'),
 '1 ¼ Glas': (1.25, 'Glas'),
 '3 Handvoll': (3, 'Handvoll'),
 '4 Scheibe/n': (4, 'Scheibe'),
 '70': (70, None),
 '555 ml': (555, 'ml'),
 '24 Kugeln': (24, 'Kugeln'),
 '37 g': (37, 'g'),
 '35 g': (35, 'g'),
 '1 ½ dl': (150.0, 'ml'),
 '127 g': (127, 'g'),
 '1300 ml': (1300, 'ml'),
 '400 große': (400, 'große'

In [12]:
with open("data/recipes/parsed_amounts.json", "w") as f:
    json.dump(parsed_amounts, f)